<a href="https://colab.research.google.com/github/DibyaSadhukhan/Amazon_Review_Analysis/blob/main/Reveiw_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 827.9 MB 1.2 MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-py3-none-any.whl size=829180942 sha256=f002b35a795bfcbe2a593e31f8f1e722099c35b186ce624ec39d09b45fd04700
  Stored in directory: /tmp/pip-ephem-wheel-cache-b8e4qhjl/wheels/11/95/ba/2c36cc368c0bd339b44a791c2c1881a1fb714b78c29a4cb8f5
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [ ]:
#mounting the google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Master Reveiw Datarame

In [ ]:
import pandas as pd
import numpy as np
import regex as re

#df.head()

In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/Laptop_Reveiw/Data/Master_reveiw_2021-10-09.csv')
df=df.drop(df.columns[0], axis=1)
df=df.drop_duplicates(subset=['Product_code','number of reveiws'], keep='first')
df = df[df['Reveiw Link'].notna()]
print(len(df))
df.head()

194


,Product_code,Product_name,Rating,number of reveiws,Price,Images,Link,Reveiw File,Reveiw Link
0,B097RKZYGS,"ASUS VivoBook 14 (2020), 14-inch (35.56 cms) F...",3.9 out of 5 stars,156,"₹31,990",https://m.media-amazon.com/images/I/716bvqzz7P...,https://www.amazon.in/ASUS-VivoBook-Integrated...,/content/gdrive/MyDrive/Amazon_product_Reveiws...,https://www.amazon.in/ASUS-VivoBook-Business-I...
1,B08C5W6PL2,HP Chromebook 14-inch (35.56 cms) Thin & Light...,3.9 out of 5 stars,766,"₹25,999",https://m.media-amazon.com/images/I/613WULNEr1...,https://www.amazon.in/HP-Chromebook-14a-na0003...,/content/gdrive/MyDrive/Amazon_product_Reveiws...,https://www.amazon.in/HP-Chromebook-14a-na0003...
2,B08HJYFZ14,"HP 15 AMD Athlon 15.6"" (39.62cms) HD Laptop (S...",3.5 out of 5 stars,483,"₹25,999",https://m.media-amazon.com/images/I/61bkuSlKF4...,https://www.amazon.in/HP-Athlon-15-6-inch-Silv...,/content/gdrive/MyDrive/Amazon_product_Reveiws...,https://www.amazon.in/HP-Athlon-15-6-inch-Silv...
3,B08HJZHTM1,HP 15 Intel Pentium Gold 6405U Processor Entry...,3.4 out of 5 stars,843,"₹25,999",https://m.media-amazon.com/images/I/71FB8qoB78...,https://www.amazon.in/HP-Pentium-Processor-15-...,/content/gdrive/MyDrive/Amazon_product_Reveiws...,https://www.amazon.in/HP-Pentium-Processor-15-...
4,B0924PSV78,RDP ThinBook 1010 - Intel Celeron Quad Core Pr...,3.4 out of 5 stars,258,"₹19,990",https://m.media-amazon.com/images/I/51C6EHkMQp...,https://www.amazon.in/RDP-ThinBook-1010-Celero...,/content/gdrive/MyDrive/Amazon_product_Reveiws...,https://www.amazon.in/RDP-ThinBook-1010-Celero...


In [ ]:
#reveiws = pd.DataFrame(columns=['Customer Name','Rating','Reveiw Title','Reveiw Text'])
reviews = pd.DataFrame(columns=['Product_code','Product_name','Customer_name','rating','review_title','review_text'])
review_file=[]
for i in range(len(df)):
  review_file.append(df.iat[i,-1])
#print(reveiw_link)
for link in review_file:
  try:
    filename=link.split('/')[3]
    f = open('/content/gdrive/MyDrive/Amazon_product_Reveiws/'+filename+'.txt',"r")
    a=f.read().split('||')
    for rev in a:
      name=rev.split('\',')[0]
      try:
        rating=rev.split('\',')[1]
      except:
        rating=""
      try:
        reveiwTitle=rev.split('\',')[-2]
      except:
        reveiwTitle=""
      comment=rev.split('\',')[-1]
      reviews = reviews.append({'Product_code' : link.split('/')[5] , 'Product_name' : link.split('/')[3] , 'Customer_name' : name.split('\'')[-1] , 'rating' : rating[2:], 'review_title' : reveiwTitle[2:], 'review_text': comment[2:-2] }, ignore_index = True)
    f.close()
  except:
    #print(link)
    print()
reviews.head()

,Product_code,Product_name,Customer_name,rating,review_title,review_text
0,B097RKZYGS,ASUS-VivoBook-Business-Integrated-M415DA-EB301T,Aubrey Khoza,2.0 out of 5 stars,The computer keeps on freezing when busy using...,The Laptop keeps on freezing whenever your usi...
1,B097RKZYGS,ASUS-VivoBook-Business-Integrated-M415DA-EB301T,Keshaw,5.0 out of 5 stars,I am loving it,Updated : 30 April 2021Still working the same ...
2,B097RKZYGS,ASUS-VivoBook-Business-Integrated-M415DA-EB301T,Steve smith,4.0 out of 5 stars,"Just an average device, should be fine for dai...",Don't expect anything magical at this price po...
3,B097RKZYGS,ASUS-VivoBook-Business-Integrated-M415DA-EB301T,Ayush Sengupta,4.0 out of 5 stars,Display Issue!,I dont know whether my device is faulty or not...
4,B097RKZYGS,ASUS-VivoBook-Business-Integrated-M415DA-EB301T,Thunderbird,4.0 out of 5 stars,Great laptop with just a couple of misses,"Laptop is good for home use. SSD, as expected..."


In [ ]:
#reviews['review_text'].replace('', np.nan, inplace=True)
#reviews = reviews[reviews['review_text'].notna()]
print(len(reviews))
reviews.to_csv('/content/gdrive/MyDrive/Laptop_Reveiw/Data/Reviews.csv')

12783


#review analysis

In [ ]:
import pandas as pd
import regex as re
import nltk
import numpy as np
nltk.download('wordnet')
from textblob import TextBlob
import spacy

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/Laptop_Reveiw/Data/Reviews.csv')
df=df.drop(df.columns[0], axis=1)
#dropping reveiws without any reveiw text 
#print(len(df))
df = df[df['review_text'].notna()]
#print(len(df))
print(df.isnull().sum())
#df.head()

Product_code     0
Product_name     0
Customer_name    0
rating           0
review_title     1
review_text      0
dtype: int64


In [ ]:
def format_text(revs):
  Non_alpha=re.compile(r'[\W]')# removing non word characters
  Non_Ascii=re.compile(r'[^a-z0-1\s]')#removing non ascii
  formatted=""
  for rev in revs.split('\n'):
    rev=rev.lower()
    rev_noPun=Non_alpha.sub(r' ',rev)
    rev=Non_Ascii.sub(r' ',rev_noPun)
    formatted+=rev
  return formatted

In [ ]:
#EDA
df=df.reset_index()
df=df.drop(df.columns[0], axis=1)
df=df.drop(df.loc[df['rating']=='Good but could have a better option for external memory'].index,axis=0)
df['rating']=df['rating'].astype('string')
#print(df['rating'].value_counts())
df['rating']=df['rating'].apply(lambda x : float(x.split(' ')[0]))
df['review_text']=df['review_text'].astype(str)
#df['rating']=df['rating'].apply(lambda x : float(x.split(' ')[0]))
#df['rating']=df['rating'].astype(str)
df.head()

,Product_code,Product_name,Customer_name,rating,review_title,review_text
0,B097RKZYGS,ASUS-VivoBook-Business-Integrated-M415DA-EB301T,Aubrey Khoza,2.0,The computer keeps on freezing when busy using...,The Laptop keeps on freezing whenever your usi...
1,B097RKZYGS,ASUS-VivoBook-Business-Integrated-M415DA-EB301T,Keshaw,5.0,I am loving it,Updated : 30 April 2021Still working the same ...
2,B097RKZYGS,ASUS-VivoBook-Business-Integrated-M415DA-EB301T,Steve smith,4.0,"Just an average device, should be fine for dai...",Don't expect anything magical at this price po...
3,B097RKZYGS,ASUS-VivoBook-Business-Integrated-M415DA-EB301T,Ayush Sengupta,4.0,Display Issue!,I dont know whether my device is faulty or not...
4,B097RKZYGS,ASUS-VivoBook-Business-Integrated-M415DA-EB301T,Thunderbird,4.0,Great laptop with just a couple of misses,"Laptop is good for home use. SSD, as expected..."


In [ ]:
#from eda we see some reveiws start with updated: date and we dont want that so we remove it
remove_updated = lambda x: re.split(r'(?:Updated :)\s\d\d\s(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}',x)[-1] if x.startswith('Updated : ') else x
df['review_text']=df['review_text'].apply(remove_updated)
#setting polarity of the reveiws
df['Polarity']=df['review_text'].apply(lambda x: TextBlob(x).sentiment.polarity)
#df['subjectivity']=df['review_text'].apply(lambda x: TextBlob(x).sentiment.subjectivity)
df.head()

,Product_code,Product_name,Customer_name,rating,review_title,review_text,Polarity
0,B097RKZYGS,ASUS-VivoBook-Business-Integrated-M415DA-EB301T,Aubrey Khoza,2.0,The computer keeps on freezing when busy using...,The Laptop keeps on freezing whenever your usi...,0.000000
1,B097RKZYGS,ASUS-VivoBook-Business-Integrated-M415DA-EB301T,Keshaw,5.0,I am loving it,Still working the same as on day 1. No issues ...,0.275159
2,B097RKZYGS,ASUS-VivoBook-Business-Integrated-M415DA-EB301T,Steve smith,4.0,"Just an average device, should be fine for dai...",Don't expect anything magical at this price po...,0.154167
3,B097RKZYGS,ASUS-VivoBook-Business-Integrated-M415DA-EB301T,Ayush Sengupta,4.0,Display Issue!,I dont know whether my device is faulty or not...,0.121131
4,B097RKZYGS,ASUS-VivoBook-Business-Integrated-M415DA-EB301T,Thunderbird,4.0,Great laptop with just a couple of misses,"Laptop is good for home use. SSD, as expected...",0.095701


In [ ]:
#setting the tone of the reveiw according to the polarity of the reveiw 
df['Tone']=None
df['Polarity']=df['Polarity'].astype('float')
df.loc[df['Polarity']>0,['Tone']]='Good'
df.loc[df['Polarity']==0,['Tone']]='Neutral'
df.loc[df['Polarity']<0,['Tone']]='Bad'
df.head()

,Product_code,Product_name,Customer_name,rating,review_title,review_text,Polarity,Tone
0,B097RKZYGS,ASUS-VivoBook-Business-Integrated-M415DA-EB301T,Aubrey Khoza,2.0,The computer keeps on freezing when busy using...,The Laptop keeps on freezing whenever your usi...,0.000000,Neutral
1,B097RKZYGS,ASUS-VivoBook-Business-Integrated-M415DA-EB301T,Keshaw,5.0,I am loving it,Still working the same as on day 1. No issues ...,0.275159,Good
2,B097RKZYGS,ASUS-VivoBook-Business-Integrated-M415DA-EB301T,Steve smith,4.0,"Just an average device, should be fine for dai...",Don't expect anything magical at this price po...,0.154167,Good
3,B097RKZYGS,ASUS-VivoBook-Business-Integrated-M415DA-EB301T,Ayush Sengupta,4.0,Display Issue!,I dont know whether my device is faulty or not...,0.121131,Good
4,B097RKZYGS,ASUS-VivoBook-Business-Integrated-M415DA-EB301T,Thunderbird,4.0,Great laptop with just a couple of misses,"Laptop is good for home use. SSD, as expected...",0.095701,Good


In [ ]:
#making the master dataset which will contain the analysis of all the products
master=pd.read_csv('/content/gdrive/MyDrive/Laptop_Reveiw/Data/Master_reveiw_2021-10-09.csv')
master=master.drop(master.columns[[0,-2]], axis=1)
master = master.rename(columns={'number of reveiws': 'number_of_reveiws'})
master['Rating']=master['Rating'].apply(lambda x : float(x.split(' ')[0]))
master=master.drop_duplicates(subset=['Product_code','number_of_reveiws'], keep='first')
master=master.reset_index(drop=True)
master['Avg_rev_rating']=None
master['rev_rat_dist']=None
master['rev_tone_dist']=None
master.head()

,Product_code,Product_name,Rating,number_of_reveiws,Price,Images,Link,Reveiw Link,Avg_rev_rating,rev_rat_dist,rev_tone_dist
0,B097RKZYGS,"ASUS VivoBook 14 (2020), 14-inch (35.56 cms) F...",3.9,156,"₹31,990",https://m.media-amazon.com/images/I/716bvqzz7P...,https://www.amazon.in/ASUS-VivoBook-Integrated...,https://www.amazon.in/ASUS-VivoBook-Business-I...,None,None,None
1,B08C5W6PL2,HP Chromebook 14-inch (35.56 cms) Thin & Light...,3.9,766,"₹25,999",https://m.media-amazon.com/images/I/613WULNEr1...,https://www.amazon.in/HP-Chromebook-14a-na0003...,https://www.amazon.in/HP-Chromebook-14a-na0003...,None,None,None
2,B08HJYFZ14,"HP 15 AMD Athlon 15.6"" (39.62cms) HD Laptop (S...",3.5,483,"₹25,999",https://m.media-amazon.com/images/I/61bkuSlKF4...,https://www.amazon.in/HP-Athlon-15-6-inch-Silv...,https://www.amazon.in/HP-Athlon-15-6-inch-Silv...,None,None,None
3,B08HJZHTM1,HP 15 Intel Pentium Gold 6405U Processor Entry...,3.4,843,"₹25,999",https://m.media-amazon.com/images/I/71FB8qoB78...,https://www.amazon.in/HP-Pentium-Processor-15-...,https://www.amazon.in/HP-Pentium-Processor-15-...,None,None,None
4,B0924PSV78,RDP ThinBook 1010 - Intel Celeron Quad Core Pr...,3.4,258,"₹19,990",https://m.media-amazon.com/images/I/51C6EHkMQp...,https://www.amazon.in/RDP-ThinBook-1010-Celero...,https://www.amazon.in/RDP-ThinBook-1010-Celero...,None,None,None


In [ ]:
master['Rating'].value_counts().sort_index(ascending=False)

5.0    12
4.8     3
4.7     4
4.6     3
4.5     9
4.4     9
4.3    20
4.2    18
4.1    14
4.0    20
3.9    14
3.8    11
3.7     8
3.6     9
3.5    11
3.4    15
3.3     8
3.2     2
3.1     3
3.0     3
2.9     1
2.8     2
2.7     2
2.4     1
2.0     2
1.6     1
1.0     1
Name: Rating, dtype: int64

In [ ]:
product_codes=list(master['Product_code'].value_counts().index)

for code in product_codes:
  reviews=df.loc[df['Product_code']==code]
  master.iat[master.loc[master['Product_code']==code].index.values[0],-3]=reviews['rating'].mean()
  master.iat[master.loc[master['Product_code']==code].index.values[0],-2]=' '.join([str(ele).strip() for ele in reviews['rating'].value_counts().sort_index(ascending=False).values])
  master.iat[master.loc[master['Product_code']==code].index.values[0],-1]=' '.join([str(ele).strip() for ele in reviews['Tone'].value_counts().sort_index(ascending=False).values])
master.head()

,Product_code,Product_name,Rating,number_of_reveiws,Price,Images,Link,Reveiw Link,Avg_rev_rating,rev_rat_dist,rev_tone_dist
0,B097RKZYGS,"ASUS VivoBook 14 (2020), 14-inch (35.56 cms) F...",3.9,156,"₹31,990",https://m.media-amazon.com/images/I/716bvqzz7P...,https://www.amazon.in/ASUS-VivoBook-Integrated...,https://www.amazon.in/ASUS-VivoBook-Business-I...,3.318182,21 17 8 2 18,10 43 13
1,B08C5W6PL2,HP Chromebook 14-inch (35.56 cms) Thin & Light...,3.9,766,"₹25,999",https://m.media-amazon.com/images/I/613WULNEr1...,https://www.amazon.in/HP-Chromebook-14a-na0003...,https://www.amazon.in/HP-Chromebook-14a-na0003...,3.257692,103 41 20 12 84,26 171 63
2,B08HJYFZ14,"HP 15 AMD Athlon 15.6"" (39.62cms) HD Laptop (S...",3.5,483,"₹25,999",https://m.media-amazon.com/images/I/61bkuSlKF4...,https://www.amazon.in/HP-Athlon-15-6-inch-Silv...,https://www.amazon.in/HP-Athlon-15-6-inch-Silv...,2.610687,31 14 13 19 54,24 60 47
3,B08HJZHTM1,HP 15 Intel Pentium Gold 6405U Processor Entry...,3.4,843,"₹25,999",https://m.media-amazon.com/images/I/71FB8qoB78...,https://www.amazon.in/HP-Pentium-Processor-15-...,https://www.amazon.in/HP-Pentium-Processor-15-...,2.305439,45 21 27 15 131,48 85 106
4,B0924PSV78,RDP ThinBook 1010 - Intel Celeron Quad Core Pr...,3.4,258,"₹19,990",https://m.media-amazon.com/images/I/51C6EHkMQp...,https://www.amazon.in/RDP-ThinBook-1010-Celero...,https://www.amazon.in/RDP-ThinBook-1010-Celero...,2.832215,45 23 9 6 66,26 83 40


In [ ]:
#product_codes[0]
master['Avg_rev_rating']=master['Avg_rev_rating'].fillna(0)
master.loc[master['Product_code']=='B09BF6S7RX']

,Product_code,Product_name,Rating,number_of_reveiws,Price,Images,Link,Reveiw Link,Avg_rev_rating,rev_rat_dist,rev_tone_dist
106,B09BF6S7RX,"ASUS VivoBook 14 (2021), Intel Core i3-1115G4 ...",4.0,8,"₹41,990",https://m.media-amazon.com/images/I/71WuDXpTAl...,https://www.amazon.in/ASUS-VivoBook-i3-1115G4-...,https://www.amazon.in/ASUS-VivoBook-i3-1115G4-...,0.0,,


In [ ]:
master.to_csv('Master_df.csv')

In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/DibyaSadhukhan/Amazon_Review_Analysis/main/Data/Master_df.csv')
a=data.iloc[0]
a=dict(a)
a['rev_rat_dist']=list(map(int,a['rev_rat_dist'].split(' ')))
#print(a['rev_rat_dist'])
#print(data.dtypes)
#det=data.loc[data['Product_code']=='yo']
#print(det)
#det=dict(det)
a['Images'].values

AttributeError: ignored

# test1

In [ ]:
def format_text(revs):
  Non_alpha=re.compile(r'[\W]')# removing non word characters
  Non_Ascii=re.compile(r'[^a-z0-1\s]')#removing non ascii
  formatted=""
  for rev in revs.split('\n'):
    rev=rev.lower()
    rev_noPun=Non_alpha.sub(r' ',rev)
    rev=Non_Ascii.sub(r' ',rev_noPun)
    formatted+=rev
  return formatted
def get_aspects(x):
    import en_core_web_lg
    nlp = en_core_web_lg.load()
    stop_words=["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", 
            "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", 
            "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", 
            "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", 
            "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", 
            "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", 
            "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", 
            "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", 
            "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than",
            "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]
    doc=nlp(x) ## Tokenize and extract grammatical components
    doc=[i.text for i in doc if i.text not in stop_words and i.pos_=="NOUN"] ## Remove common words and retain only nouns
    doc=list(map(lambda i: i.lower(),doc)) ## Normalize text to lower case
    doc=pd.Series(doc)
    counts=doc.value_counts().head(10).tolist()
    doc=doc.value_counts().head(10).index.tolist() ## Get 5 most frequent nouns
    return doc,counts
def get_polarity(reveiws):
  reveiws['Reveiw Text']= reveiws['Reveiw Text'].astype(str)
  pol = lambda x: TextBlob(x).sentiment.polarity
  reveiws['polarity']= reveiws['Reveiw Text'].apply(pol)
  return reveiws
def get_Reveiw_df(filename):
  f = open('/content/gdrive/MyDrive/Amazon_product_Reveiws/'+filename+'.txt',"r")
  a=f.read().split('||')
  reveiws = pd.DataFrame(columns=['Customer Name','Rating','Reveiw Title','Reveiw Text'])
  for rev in a:
    name=rev.split('\',')[0]
    try:
      rating=rev.split('\',')[1]
    except:
      rating=""
    try:
      reveiwTitle=rev.split('\',')[-2]
    except:
      reveiwTitle=""
    comment=rev.split('\',')[-1]
    reveiws = reveiws.append({'Customer Name' : name.split('\'')[-1] , 'Rating' : rating[2:], 'Reveiw Title' : reveiwTitle[2:], 'Reveiw Text': comment[2:-2] }, ignore_index = True)
  f.close()
  #reveiws.head()
  return reveiws
def get_polarity_txt(list):
  pol = lambda x: TextBlob(x).sentiment.polarity
  return map(pol,list)


In [ ]:
reveiw_link=[]
for i in range(len(df)):
  reveiw_link.append(df.iat[i,-6])
print(reveiw_link)

['https://www.amazon.in/ASUS-VivoBook-Business-Integrated-M415DA-EB301T/product-reviews/B097RKZYGS/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews', 'https://www.amazon.in/HP-Chromebook-14a-na0003TU-Touchscreen-Expandable/product-reviews/B08C5W6PL2/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews', 'https://www.amazon.in/HP-Athlon-15-6-inch-Silver-3050U-15s-gy0001au/product-reviews/B08HJYFZ14/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews', 'https://www.amazon.in/HP-Pentium-Processor-15-6-inch-15s-du1052tu/product-reviews/B08HJZHTM1/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews', 'https://www.amazon.in/RDP-ThinBook-1010-Celeron-Processor/product-reviews/B0924PSV78/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews', 'https://www.amazon.in/Lenovo-15-6-inch-Integrated-Graphics-82C700J0IH/product-reviews/B095JY5B3F/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews', 'https://www.amazon.in/Lenovo-Integrated-Graphics

In [ ]:
df_index=0
for links in reveiw_link[:3]:
  try:
    filename=links.split('/')[3]
    reveiws=get_Reveiw_df(filename)
    reveiws['Actual Rating']=reveiws['Rating'].apply(lambda x : (x.split(' ')[0].strip()))
    reveiws['Actual Rating']=pd.to_numeric(reveiws['Actual Rating'], downcast='float')
    for index,row in reveiws.iterrows():
      revt=reveiws.iat[index,-2]
      #print(revt)
      flag=re.findall(r'(?:Updated :)\s\d\d\s(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}',revt)
      if len(flag)>0:
        comt=revt.split(flag[0])[-1]
        reveiws.iat[index,-2]=comt
    reveiws=get_polarity(reveiws)
    reveiws['polarity']=reveiws['polarity'].astype('float')
    reveiws['Tone']=None
    reveiws.loc[reveiws['polarity']>0,['Tone']]='Good'
    reveiws.loc[reveiws['polarity']==0,['Tone']]='Neutral'
    reveiws.loc[reveiws['polarity']<0,['Tone']]='Bad'
    
    rev = '\n'.join(reveiws["Reveiw Text"])
    all_rev= format_text(rev)
    a=reveiws['Tone'].value_counts()
    df.iat[df_index,-4]=str([a.index.to_list(),a.values.tolist()])
    #print(rev)
    aspects,counts=get_aspects(all_rev)
    aspect_sent=[]
    for i in range(len(aspects)):
      if counts[i]>sum(counts)/len(counts):
        aspect_sentence=re.findall(r'([^.]*'+aspects[i].lower()+'[^.]*)', rev.lower())
        aspect_sent.append(format_text(aspect_sentence))
    df.iat[df_index,-2]=aspect_sent
    df.iat[df_index,-3]=[aspects,counts]
    df.iat[df_index,-5]=reveiws['Actual Rating'].mean()
    df_index+=1
  except:
    df_index+=1
#reveiws.dtypes
df.head(5)

,Unnamed: 0,Product_code,Product_name,Rating,number of reveiws,Price,Images,Link,Reveiw File,Reveiw Link,Avg Reveiw Rating,Tone,aspects and counts,aspect Sentences,aspect Sentence polarity
0,0,B097RKZYGS,"ASUS VivoBook 14 (2020), 14-inch (35.56 cms) F...",3.9,156,"₹31,990",https://m.media-amazon.com/images/I/716bvqzz7P...,https://www.amazon.in/ASUS-VivoBook-Integrated...,/content/gdrive/MyDrive/Amazon_product_Reveiws...,https://www.amazon.in/ASUS-VivoBook-Business-I...,0.0,"[['Good', 'Bad', 'Neutral'], [43, 13, 11]]","[['Good', 'Bad', 'Neutral'], [43, 13, 11]]",,
1,1,B08C5W6PL2,HP Chromebook 14-inch (35.56 cms) Thin & Light...,3.9,766,"₹25,999",https://m.media-amazon.com/images/I/613WULNEr1...,https://www.amazon.in/HP-Chromebook-14a-na0003...,/content/gdrive/MyDrive/Amazon_product_Reveiws...,https://www.amazon.in/HP-Chromebook-14a-na0003...,0.0,"[['Good', 'Bad', 'Neutral'], [171, 63, 29]]","[['Good', 'Bad', 'Neutral'], [171, 63, 29]]",,
2,2,B08HJYFZ14,"HP 15 AMD Athlon 15.6"" (39.62cms) HD Laptop (S...",3.5,483,"₹25,999",https://m.media-amazon.com/images/I/61bkuSlKF4...,https://www.amazon.in/HP-Athlon-15-6-inch-Silv...,/content/gdrive/MyDrive/Amazon_product_Reveiws...,https://www.amazon.in/HP-Athlon-15-6-inch-Silv...,0.0,"[['Good', 'Bad', 'Neutral'], [60, 47, 28]]","[['Good', 'Bad', 'Neutral'], [60, 47, 28]]",,
3,3,B08HJZHTM1,HP 15 Intel Pentium Gold 6405U Processor Entry...,3.4,843,"₹25,999",https://m.media-amazon.com/images/I/71FB8qoB78...,https://www.amazon.in/HP-Pentium-Processor-15-...,/content/gdrive/MyDrive/Amazon_product_Reveiws...,https://www.amazon.in/HP-Pentium-Processor-15-...,0.0,"[['Bad', 'Good', 'Neutral'], [106, 85, 49]]","[['Bad', 'Good', 'Neutral'], [106, 85, 49]]",,
4,4,B0924PSV78,RDP ThinBook 1010 - Intel Celeron Quad Core Pr...,3.4,258,"₹19,990",https://m.media-amazon.com/images/I/51C6EHkMQp...,https://www.amazon.in/RDP-ThinBook-1010-Celero...,/content/gdrive/MyDrive/Amazon_product_Reveiws...,https://www.amazon.in/RDP-ThinBook-1010-Celero...,0.0,"[['Good', 'Bad', 'Neutral'], [83, 40, 29]]","[['Good', 'Bad', 'Neutral'], [83, 40, 29]]",,


In [ ]:
df.to_csv('hi.csv')

NameError: ignored

In [ ]:
#print(len(reveiws))
a=reveiws['Tone'].value_counts()
df['Tone']=[a.index.to_list(),a.values.tolist()])

[['Good', 'Bad', 'Neutral'], [40, 10, 8]]


In [ ]:
reveiws.head()

,Customer Name,Rating,Reveiw Title,Reveiw Text,polarity,Tone
0,Aubrey Khoza,2.0 out of 5 stars,The computer keeps on freezing when busy using...,The Laptop keeps on freezing whenever your usi...,0.000000,Neutral
1,Keshaw,5.0 out of 5 stars,I am loving it,Still working the same as on day 1. No issues ...,0.275159,Good
2,Steve smith,4.0 out of 5 stars,"Just an average device, should be fine for dai...",Don't expect anything magical at this price po...,0.154167,Good
3,Ayush Sengupta,4.0 out of 5 stars,Display Issue!,I dont know whether my device is faulty or not...,0.121131,Good
4,Thunderbird,4.0 out of 5 stars,Great laptop with just a couple of misses,"Laptop is good for home use. SSD, as expected...",0.095701,Good


In [ ]:
reveiws['Tone'].value_counts()


Good       40
Bad        10
Neutral     8
Name: Tone, dtype: int64

In [ ]:
rev = '\n'.join(reveiws["Reveiw Text"])
rev= format_text(rev)
#print(rev)
print(get_aspects(rev))


the laptop keeps on freezing whenever your using it  how can  i resolve the problem still working the same as on day 1  no issues so far  update on battery life  i was wrong  it goes     hrs easily  if you go for another laptop  make sure that have ssd  it does magical performance improvements and increase battery life          this seems to be the best laptop below  0k  got in    00   absolutely premium look and performance 1  display is above average but not the best per fhd configuration    performance is awesome  mainly due to ryzen   and ssd memory    fingerprint sensor is perfect    very light weight and portable  size is perfect    sound is awesome    battery goes up to     hrs easily cons  power button is placed beside the delete key within keyboard  many times you go for delete but shutdown the laptop  i just disabled the action of power button from windows settings so nothing happen if i press power button accidentally  no issues now overall good for home office use and can b

# test code

In [ ]:
aspects,counts=get_aspects(rev)

In [ ]:
#print([aspects,counts])
#reveiws['aspect_sentence']=reveiws['Reveiw Text'].apply(lambda x : pick_only_key_sentence(x,aspects[:5]))
rev = '\n'.join(reveiws["Reveiw Text"])
aspect_sent=[]
for i in range(len(aspect)):
  if counts[i]>sum(counts)/len(counts):
    aspect_sent.append(re.findall(r'([^.]*'+aspects[i].lower()+'[^.]*)', rev.lower()))
print(aspect_sent[0])


['the laptop keeps on freezing whenever your using it', ' if you go for another laptop, make sure that have ssd', '---------this seems to be the best laptop below 30k (got in 28500)', ' many times you go for delete but shutdown the laptop', " it's an average laptop meant for average office work or study purposes", " the laptop may seem laggy at the beginning, that's because of the pre-installed mcafee antivirus, just uninstall it and you should be fine", ' when i turn on the laptop everything is fine', ' if i restart the laptop, then everything becomes fine again', '\nlaptop is good for home use', '  laptop does have a good battery life atleast to last 4-5 hours with good use', "4 (silly) things that the laptop could've done better with -1", '  within a couple of days, i  accidentally put the laptop to sleep atleast for 5 times', 'overall a great laptop for the price (29k) and none of the cons are deal readers for me', '\nthis is budget laptop so there isn’t point talking about gaming'

In [ ]:
revs1=[]
revs1.append(reveiws['aspect_sentence'])
def merge_aspect_sent(revs1):
  aspect_sent=""
  for rev in revs1[0]:
      aspect_sent=aspect_sent+'\n'.join(rev)
  return aspect_sent
a=merge_aspect_sent(revs1)
print(a)

 update on battery life, i was wrong
 it does magical performance improvements and increase battery life
 battery goes up to 4-5 hrs easily  laptop does have a good battery life atleast to last 4-5 hours with good use#battery: 4/5
 first of all this model m415da comes with 37whrs(m409da comes with 32whrs & mi notebook el comes with 46whrs) battery capacity
 battery backup is at least 4-5 hrs long battery lifecons:1 battery is decent, could have been better, camera is average and battery 3hours working normal uses and design good performance good boot time is also goodyou can and on it after ram and storage upgradation but don't recommend to game on itother wise beast in small bugdetbattery is average of 4 to 5 will surfing internet and 1nice laptop, good battery ok for normal work, light gaming can do internet surfing and more
 average battery life 4 to 6 hours when on wifi, but upto 10 hours when offline3) battery performance is very low this product is good but there is one mistake i

In [ ]:
reveiws.to_csv("hi.csv")